In [1]:
import math
import webdataset as wds
import scipy.ndimage as ndi
import numpy as np
import matplotlib.pyplot as plt
from itertools import islice
import ray
from ray.data import Dataset
import braceexpand
import requests
import random
import json
import os
import torch
import sys

if "GOOGLE_APPLICATION_CREDENTIALS" in os.environ:
    del os.environ["GOOGLE_APPLICATION_CREDENTIALS"]

# ray.init()

In [ ]:
source = list(braceexpand.braceexpand("gs://ocro-arxiv/pages/arxiv-pages-{000000..001038}.tar"))
destination = "gs://ocro-tempout/shuffled"

!gsutil -m rm gs://ocro-tempout/shuffled/*.tar
def run(cmd):
    print("#", cmd, file=sys.stderr)
    assert os.system(cmd) == 0
def print_sample(sample):
    for k, v in sample.items():
        print(k, type(v), repr(v)[:50])
ds = wds.WebDataset(source, handler=wds.ignore_and_continue)
dl = wds.WebLoader(ds, num_workers=16, batch_size=None).shuffle(20000)
def upload(fname):
    run(f"sync; sleep 1; gsutil --quiet -m cp {fname} {destination}/{fname}")
    os.unlink(fname)
sink = wds.ShardWriter("arxiv-shuffled-%06d.tar", maxcount=1000, maxsize=500e6, post=upload, encoder=None)
for i, sample in enumerate(dl):
    if i % 1000 == 0:
        print(i)
    sink.write(sample)
sink.close()

In [ ]:
#! gsutil -m cp gs://ocro-tempout/shuffled/arxiv-shuffled-*.tar gs://ocro-arxiv/shuffled/

In [ ]:
shards = os.popen("gsutil ls gs://ocro-tempout/shuffled").readlines()
shards = [s.strip() for s in shards]
shards = [s for s in shards if s.endswith(".tar")]
allshards = braceexpand.braceexpand("gs://ocro-tempout/shuffled/arxiv-shuffled-{000000..007423}.tar")
set(allshards) - set(shards)

In [6]:
shards = os.popen("gsutil ls gs://ocro-arxiv/shuffled").readlines()
shards = [s.strip() for s in shards]
shards = [s for s in shards if s.endswith(".tar")]
allshards = braceexpand.braceexpand("gs://ocro-arxiv/shuffled/arxiv-shuffled-{000000..007423}.tar")
set(allshards) - set(shards)

set()